<a href="https://colab.research.google.com/github/rhj2000/rhj2000.github.io/blob/master/Copy_of_Setup_Genga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup of GENGA on Google Colab

[GENGA](https://bitbucket.org/sigrimm/genga/src/master/) is a code that uses GPUs to perform N-body simulations (see the [docs](https://genga.readthedocs.io/en/latest/)).  This tutorial will show you how to setup GENGA using the GPU resources of Google Colab.  First, you must ensure that you have access to a GPU through the follwing:

- Go to Runtime-->Change runtime type
- Select GPU from a dropdown menu

The source code for GENGA needs to loaded into Google Colab.  You can mount your Google Drive to have persistent memory or use the local memory for Google Colab, which is cleared after your session.  For the latter, you must be sure to download your results.  After uploading the GENGA source zip file (or using wget), you can use "!unzip" to extract the source in to the local directory.  

- Note: It is easier to manage if you rename the directory of the extracted files to "genga" or something shorter.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -O sigrimm-genga.zip https://bitbucket.org/sigrimm/genga/get/31da424f602d.zip
!unzip sigrimm-genga.zip

--2022-01-27 05:30:44--  https://bitbucket.org/sigrimm/genga/get/31da424f602d.zip
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3492243 (3.3M) [application/zip]
Saving to: ‘sigrimm-genga.zip’

sigrimm-genga.zip   100%[===================>]   3.33M  9.37MB/s    in 0.4s    

2022-01-27 05:30:45 (9.37 MB/s) - ‘sigrimm-genga.zip’ saved [3492243/3492243]

Archive:  sigrimm-genga.zip
31da424f602d99cdd13ad892ccbf12b5a8ae4883
   creating: sigrimm-genga-31da424f602d/
  inflating: sigrimm-genga-31da424f602d/.gitignore  
  inflating: sigrimm-genga-31da424f602d/.readthedocs.yaml  
  inflating: sigrimm-genga-31da424f602d/Documentation.md  
   creating: sigrimm-genga-31da424f602d/GengaGL/
  inflating: sigrimm-genga-31da424f602d/GengaGL/Makefile  
  inflating: sigrimm-genga-31da424f602d/GengaGL/gengaGL

After extracting the files and renaming the directory, we need to move to the source directory using "%cd /content/genga/source/"

In [ ]:
%cd /content/genga/source

/content/genga/source


While in the source directory, we need to first check the type of GPU is on the hosted runtime.  This could be an Nvidia K80, T4, or P100, where each of these cards have a different compute capability (which determines how it is compiled in CUDA).  You can use the "!nvidia-smi" command to determine which card is available.  Then GENGA can be compiled using the "!make SM=XX" command, where the XX is replaced with the compute capability (e.g., 3.5-->35).

Next you need to move to the directory with the input files for GENGA.  There are only 2 files:

- initialdisk.dat
    - contains the initial states of all the particles
- param.dat
    - configures the settings for the simulation
    - some settings are how long to run the simulations or how frequent you want to have output


In [31]:
!nvidia-smi

Mon Jan 31 20:54:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
!rm genga #use to remake genga after building with wrong cuda version

In [ ]:
!git init

Initialized empty Git repository in /content/genga/source/.git/


In [34]:
!make SM=37

nvcc -arch sm_37 --compiler-options -Wall -o genga genga.cu Host2.cu Orbit2.cu Energy.cu output.cu integrator.cu gas.cu  -DGIT_BRANCH=\"`git rev-parse --abbrev-ref HEAD`\" -DGIT_COMMIT=\"`git rev-parse HEAD`\" -DBUILD_DATE=\""`date`"\" -DBUILD_SYSTEM=\""`uname -a`"\" -DBUILD_PATH=\""`pwd`"\" -DBUILD_SM=\"37\"
fatal: ambiguous argument 'HEAD': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
fatal: ambiguous argument 'HEAD': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [35]:
!mkdir ../../temp

mkdir: cannot create directory ‘../../temp’: File exists


In [36]:
%cd ../../temp/

/content/temp


In [ ]:
!cp ../genga/source/param.dat .

In [ ]:
!cp ../genga/source/initialplanet_2048.dat .

Finally, we can run GENGA.  Since we are in the "temp" directory, we use the relative path to the genga executable that we compiled.

In [37]:
!../genga/source/genga

Start GENGA
Read parameters
Parameters OK
Read Size
Size OK

 ******** Simulation path  ********

Genga Version: 3.137
Mercurial Branch: HEAD
Mercurial Commit: HEAD
Build Date: Mon Jan 31 20:54:46 UTC 2022
Build Path: /content/genga/source
Build System: Linux c5cc844b306c 5.4.144+ #1 SMP Tue Dec 7 09:58:10 PST 2021 x86_64 x86_64 x86_64 GNU/Linux
Build Compute Capability: SM=37
Do Kick in single precision: 0
Serial Grouping: 0
Do kernel tuning: 1
Compute Poincare Section: 0
FormatS: 0
FormatT: 0
FormatP: 1
FormatO: 0
NmaxM: 32
Time step in days: 6 
Starting time: 0 
Output name: test
Energy output interval: 100
Coordinates output interval: 100
Number of outputs per interval: 1
Coordinate output buffer: 1
Use Irregular outputs: 0
Irregular output calendar: -
Use Transits: 0
Use RV: 0
TTV file name: -
RV file name: -
Print Transits: 0
Print RV: 0
Print MCMC: 0
MCMC NE: 0
MCMC Restart: 0
Integration steps: 100000
Central Mass: 1
Star Radius: 0.00465476
Star Love Number: 1
Star fluid Love N